In [2]:
import os
import sys

# Explicitly set the project root
project_root = '/Users/tayebekavousi/Desktop/sentimentAnalysisPortfolio'
# Add to Python path
sys.path.insert(0, project_root)
os.chdir('../')  # Moving up one directory to the root

In [3]:
from main import SentimentAnalyzer
from config.model_config import ModelConfig 
from models.modelPersistence import ModelPersistence
from utils.modelEvaluator import ModelEvaluator

2024-12-12 11:28:00.771606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Transformers package location: /opt/anaconda3/lib/python3.12/site-packages/transformers/__init__.py


In [4]:
modelPersistence = ModelPersistence()
analyzer = SentimentAnalyzer()

In [5]:
# Create and setup analyzer
analyzer.process_data()

Training set size: 2100 (70.0%)
Validation set size: 600 (20.0%)
Test set size: 300 (10.0%)


(array(["Came here looking for a tablet. JUST wanted to look around and see what was available. Even if I did find what I was looking for, there would have been ABSOLUTELY NO WAY I would have purchased it here. I was approached and asked if I needed any help/assistance 5 times in less than 7 minutes.  Three of those times were by the same women. Seriously?!?! You people are worse than a used car lot. By the 3rd time being approached, I just wanted to get the hell out of there. I had no interest in looking or being there in general. Its one thing to be approached once or twice, but FIVE times in less than 7 minutes is down right ridiculous!\\nWhile I was there, I did have to use the women's restroom and WOW!!!!! Ive been to run down nasty gas stations with cleaner bathrooms than this place. I honestly couldnt believe how dirty it was. Obviously the only people that actually work there are the absolutely annoying tablet sales people!",
        "1 Star for the mixed up order, slow service

In [6]:
# 3. loading the previously saved model:
epoch = 2  # or whatever epoch the model is saved
# model_dir = os.path.join(os.getcwd(), 'saved_models')
loaded_model, training_history = modelPersistence.load_saved_model(epoch)

Loading model from directory: /Users/tayebekavousi/Desktop/sentimentAnalysisPortfolio/saved_models
Loaded model configuration and training history
Initialized tokenizer


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Initialized model architecture

Error loading model: Failed to load model weights: Received incompatible tensor with shape (160, 3) when attempting to restore variable with shape (80, 3) and name enhanced_distil_bert_for_sentiment/sentiment/kernel:0.
Load location attempted:  /Users/tayebekavousi/Desktop/sentimentAnalysisPortfolio/saved_models


RuntimeError: Failed to load model weights: Received incompatible tensor with shape (160, 3) when attempting to restore variable with shape (80, 3) and name enhanced_distil_bert_for_sentiment/sentiment/kernel:0.

In [ ]:
# Create components

from models.sentiment_model import ModelTrainer
model = loaded_model
trainer = ModelTrainer(model=model)
evaluator = ModelEvaluator(model, trainer)

# Run evaluation
eval_results = evaluator.evaluate_model(analyzer.test_texts, analyzer.test_labels)
evaluator.visualize_performance(eval_results, training_history)

In [ ]:
from transformers import DistilBertTokenizer
from main import SentimentAnalyzer# Create tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Initialize analyzer with both model and tokenizer
analyzer = SentimentAnalyzer()
analyzer.model = loaded_model
analyzer.tokenizer = tokenizer

In [ ]:
def test_sentiment_edge_cases(analyzer):
   """
   Test sentiment analyzer with various edge cases organized by sentiment and linguistic pattern.
   
   Args:
       analyzer: Initialized SentimentAnalyzer instance
   """
   test_cases = {
       "Positive with Negation": [
           ("I can't deny that this place is amazing. Not a single thing wrong with the food or service!", "Double Negative -> Positive"),
           ("Never had a bad experience here. The food isn't anything less than perfect.", "Negated Negative"),
           ("Not once have I been disappointed by their service. The staff isn't unfriendly at all.", "Complex Negation")
       ],
       
       "Positive with Sarcasm": [
           ("Yeah right, like this restaurant could possibly get ANY better! *chef's kiss* Absolutely perfect!", "Exaggerated Praise"),
           ("Oh sure, just RUIN my diet with your impossibly delicious desserts! How dare you be this good!", "Mock Complaint"),
           ("Just what I needed - another restaurant to be obsessed with! 🙄 Now I'll have to keep coming back!", "Fake Annoyance")
       ],
       
       "Positive with Multipolarity": [
           ("The wait was long but honestly worth every minute. Amazing food and exceptional service!", "Contrast Resolution"),
           ("Small portions and pricey, but the taste makes up for everything. Will definitely return!", "Trade-off Acceptance"),
           ("Noisy atmosphere but incredible food and the best service I've had in years.", "Mixed with Positive Dominance")
       ],
       
       "Negative with Negation": [
           ("The food isn't good at all. Not worth the price and I won't be returning.", "Direct Negative"),
           ("I couldn't find anything special about this place. The service wasn't even close to acceptable.", "Compound Negative"),
           ("Not once did they get our order right. The manager wasn't helpful either.", "Sequential Negative")
       ],
       
       "Negative with Sarcasm": [
           ("Oh fantastic, another overpriced meal with cold food. Just what I was hoping for! 🙄", "Mock Enthusiasm"),
           ("Wow, amazing how they consistently manage to mess up a simple order. Such talent! 😒", "Ironic Praise"),
           ("Five stars for teaching me the true meaning of patience! Best 2-hour wait ever! 🙃", "Exaggerated Compliment")
       ],
       
       "Negative with Multipolarity": [
           ("Great location but terrible food and even worse service. Definitely not returning.", "Location vs Experience"),
           ("Beautiful decor, shame about the rude staff and inedible food.", "Aesthetics vs Function"),
           ("Nice ambiance but overpriced food and disappointing service ruined the experience.", "Environment vs Service")
       ],
       
       "Neutral with Negation": [
           ("The food isn't amazing but isn't terrible either. Just an average experience.", "Balanced Negation"),
           ("Not the best, not the worst. Wouldn't go out of my way to return.", "Double Neutral Negation"),
           ("Can't say it was great, can't say it was bad. Just okay.", "Negated Extremes")
       ],
       
       "Neutral with Sarcasm": [
           ("Well, that was... an experience. I guess that's one way to run a restaurant! 🤔", "Ambiguous Evaluation"),
           ("'Interesting' take on Italian food. Very... unique interpretation! 😏", "Noncommittal Sarcasm"),
           ("Such a... memorable experience. Definitely different from what I expected! 🫤", "Understated Sarcasm")
       ],
       
       "Neutral with Multipolarity": [
           ("Good food but slow service. Bad parking but nice location. Evens out I guess.", "Balanced Trade-offs"),
           ("Some dishes were great, others terrible. Service varied. Hard to form an opinion.", "Mixed Experience"),
           ("Excellent appetizers, mediocre mains, poor desserts. Averages out to okay.", "Quality Variation")
       ]
   }
   
   for category, cases in test_cases.items():
       print(f"\n=== Testing {category} ===\n")
       for text, case_type in cases:
           prediction = analyzer.predict(text)
           print(f"Case Type: {case_type}")
           print(f"Text: {text}")
           print(f"Prediction: {prediction}")
           print()

# Usage
test_sentiment_edge_cases(analyzer)